In [ ]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import math

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric

from  utils import accuracy, make_uniform_schedule
from models import iterativeGCN, MLP_GCN, normalNN, iterativeNN, GCN, iterativeGCN_variant
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [ ]:
dataset = Planetoid(root='data/Planetoid', 
                        name="Cora", 
                        transform=NormalizeFeatures())
data = dataset[0]

In [ ]:
torch.count_nonzero(data.x)

In [ ]:
torch.prod(torch.Tensor([data.x.shape]))

In [ ]:
hid_dim = 16
num_iter_layers = 5
dropout = 0.5
smooth_fac= 0.7

In [ ]:
mlp_gcn = MLP_GCN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_layers=num_iter_layers,
                  dropout=dropout,
                  xavier_init=True)
norm_nn = normalNN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_layers=num_iter_layers,
                  dropout=dropout,
                  xavier_init=True)
iter_gcn = iterativeGCN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_train_iter=num_iter_layers,
                  smooth_fac=smooth_fac,
                  dropout=dropout,
                  xavier_init=True)
iter_nn = iterativeNN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_train_iter=num_iter_layers,
                  smooth_fac=smooth_fac,
                  dropout=dropout,
                  xavier_init=True)
iter_nn = iterativeNN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_train_iter=num_iter_layers,
                  smooth_fac=smooth_fac,
                  dropout=dropout,
                  xavier_init=True)
gcn = GCN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=hid_dim,
                  num_layers=num_iter_layers,
                  dropout=dropout)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
gcn = GCN(input_dim=dataset.num_features,
                  output_dim=dataset.num_classes,
                  hidden_dim=32,
                  num_layers=3,
                  dropout=0.5)

In [ ]:
print("usual gcn: ", count_parameters(gcn))

In [ ]:
print("iterative NN: ", count_parameters(iter_nn))
print("normal NN: ", count_parameters(norm_nn))
print("iterative GCN: ", count_parameters(iter_gcn))
print("usual gcn: ", count_parameters(gcn))
print("gcn with mlp: ", count_parameters(mlp_gcn))

In [ ]:
print(gcn)

In [ ]:
import numpy as np
full_schedule = np.full(10, 0.5)
print("full schedule: ", full_schedule)
for iter in range(len(full_schedule)):
    eval_schedule = full_schedule[0:iter]
    print("eval schedule: ", eval_schedule)

In [ ]:
aa = F.sigmoid(torch.Tensor(np.linspace(1, 9, 30)))

In [ ]:
len(aa)

In [ ]:
import numpy as np
import torch.nn.functional as F
import torch
F.sigmoid(torch.Tensor(np.linspace(0.5, 4, 60)))

In [ ]:
F.sigmoid(torch.Tensor(np.linspace(0.5, 3, 60)))

In [ ]:
F.sigmoid(torch.Tensor([3]))

In [ ]:
from models import GCN
gcn = GCN(input_dim=1433,
                output_dim=7,
                hidden_dim=32,
                num_layers=10,
                dropout=0.5)
print(gcn)

In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import math

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric

from  utils import accuracy, make_uniform_schedule
from models import iterativeGCN, MLP_GCN, normalNN, iterativeNN, GCN, iterativeGCN_variant
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', 
                        name="Cora", 
                        transform=NormalizeFeatures())
data = dataset[0]

train_schedule = make_uniform_schedule(2, 0.7)
model = iterativeGCN_variant(input_dim=1433,
                                 output_dim=7,
                                 hidden_dim=32,
                                 train_schedule=train_schedule,
                                 dropout=0.5,
                                 xavier_init=True
                                 )
model(data.x, data.edge_index)

/Users/sizhuang/Desktop/GitHub Repos/IterativeMethods/iterENV/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


guagua
lala
lala


tensor([[-1.9445, -1.9470, -1.9416,  ..., -1.9473, -1.9455, -1.9443],
        [-1.9437, -1.9460, -1.9442,  ..., -1.9457, -1.9500, -1.9416],
        [-1.9467, -1.9464, -1.9423,  ..., -1.9472, -1.9493, -1.9440],
        ...,
        [-1.9527, -1.9442, -1.9372,  ..., -1.9402, -1.9535, -1.9451],
        [-1.9456, -1.9451, -1.9434,  ..., -1.9469, -1.9479, -1.9442],
        [-1.9449, -1.9466, -1.9434,  ..., -1.9466, -1.9458, -1.9453]],
       grad_fn=<LogSoftmaxBackward0>)